<a href="https://colab.research.google.com/github/yimei910110/gitlearn/blob/master/dl_exp8_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

class Bottleneck(nn.Module):
    expansion = 4
    
    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [2]:
class ResNet50(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet50, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(Bottleneck, 64,  3, stride=1)
        self.layer2 = self._make_layer(Bottleneck, 128, 4, stride=2)
        self.layer3 = self._make_layer(Bottleneck, 256, 6, stride=2)
        self.layer4 = self._make_layer(Bottleneck, 512, 3, stride=2)
        self.linear = nn.Linear(512*Bottleneck.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./', train=True,  download=True, transform=transform_train)
testset  = torchvision.datasets.CIFAR10(root='./', train=False, download=True, transform=transform_test)
trainsize=int(0.8*len(trainset))
valsize=len(trainset)-trainsize
trainset1,valset=torch.utils.data.random_split(trainset,[trainsize,valsize])

trainloader = torch.utils.data.DataLoader(trainset1, batch_size=128, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print('device:',device)
print('trainset,valset,testset len:',len(trainset1),len(valset),len(testset))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified
device: cuda:0
trainset,valset,testset len: 40000 10000 10000


In [4]:
# 网络放到GPU上
net = ResNet50().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [5]:
net.train()
for epoch in range(10):  # 重复多轮训练
    total_corr=0
    val_total_corr=0
    for i, (inputs, labels) in enumerate(trainloader):
        #print('inputs shape:',inputs.shape)
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 正向传播 + 反向传播 + 优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = outputs.max(1)
        num_correct = (predicted == labels).sum().item()
        total_corr += num_correct
        # 输出统计信息
        if i % 100 == 0:
            print('Epoch: %d Minibatch: %5d loss: %.3f' %(epoch + 1, i + 1, loss.item()))

    for i, (inputs, labels) in enumerate(valloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 正向传播 +　反向传播 + 优化
        outputs = net(inputs)
        val_loss = criterion(outputs, labels)
        # 输出统计信息
        _, predicted = outputs.max(1)
        num_correct = (predicted == labels).sum().item()
        val_total_corr += num_correct

    print('Epoch: %d  trainset total_corr:%i  trainset: trainset:%i total_corr:%i  valset:%i' % (epoch + 1, total_corr, len(trainset1), val_total_corr, len(valset)))
    print('Epoch: %d  train-loss: %.3f  train-accuarcy: %.3f  val-loss: %.3f  val-accuarcy: %.3f' % (epoch + 1, loss.item(), total_corr / len(trainset1),val_loss.item(),val_total_corr / len(valset)))


Epoch: 1 Minibatch:     1 loss: 2.716
Epoch: 1 Minibatch:   101 loss: 1.816
Epoch: 1 Minibatch:   201 loss: 1.661
Epoch: 1 Minibatch:   301 loss: 1.401
Epoch: 1  trainset total_corr:14695  trainset: trainset:40000 total_corr:5106  valset:10000
Epoch: 1  train-loss: 1.465  train-accuarcy: 0.367  val-loss: 1.691  val-accuarcy: 0.511
Epoch: 2 Minibatch:     1 loss: 1.325
Epoch: 2 Minibatch:   101 loss: 1.371
Epoch: 2 Minibatch:   201 loss: 1.248
Epoch: 2 Minibatch:   301 loss: 1.202
Epoch: 2  trainset total_corr:22254  trainset: trainset:40000 total_corr:5911  valset:10000
Epoch: 2  train-loss: 1.015  train-accuarcy: 0.556  val-loss: 1.119  val-accuarcy: 0.591
Epoch: 3 Minibatch:     1 loss: 1.193
Epoch: 3 Minibatch:   101 loss: 1.044
Epoch: 3 Minibatch:   201 loss: 1.026
Epoch: 3 Minibatch:   301 loss: 0.704
Epoch: 3  trainset total_corr:25888  trainset: trainset:40000 total_corr:6704  valset:10000
Epoch: 3  train-loss: 0.790  train-accuarcy: 0.647  val-loss: 1.823  val-accuarcy: 0.670
E

In [6]:
net.eval()
test_total_corr=0
for i, (inputs, labels) in enumerate(testloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    # 优化器梯度归零
    optimizer.zero_grad()
    # 正向传播 +　反向传播 + 优化
    outputs = net(inputs)
    test_loss = criterion(outputs, labels)
    # 输出统计信息
    _, predicted = outputs.max(1)
    num_correct = (predicted == labels).sum().item()
    test_total_corr += num_correct

print('testset total_corr:%i  testset:%i' % (test_total_corr, len(testset)))
print('test-loss: %.3f  test-accuarcy: %.3f ' % (test_loss.item(), test_total_corr / len(testset)))


testset total_corr:8349  testset:10000
test-loss: 0.430  test-accuarcy: 0.835 
